# import library

In [1]:
from functools import partial

import numpy as np
import pandas as pd
import os
import random
import time

import tensorflow as tf, re, math
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model, Sequential
from tensorflow.keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
import gc
import uproot

### Checking for TPUs

In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()
    
!nvidia-smi

Not connected to a TPU runtime. Using CPU/GPU strategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
/usr/bin/sh: nvidia-smi: command not found


### Loading the data

In [3]:
def get_df(root_file_name, filter_name):
    events = uproot.open(root_file_name, filter_name=filter_name)["tree"]
    df = events.arrays(library="pd")
    return df

features = []
# variables: general
features += ['FatJet_pt', 'FatJet_eta', 'FatJet_phi', 'FatJet_DDX_jetNSecondaryVertices', \
             'FatJet_DDX_jetNTracks', 'FatJet_DDX_z_ratio', 'FatJet_Proba', 'FatJet_area', \
             'FatJet_jetId', 'FatJet_lsf3', 'FatJet_rawFactor', 'FatJet_n2b1', 'FatJet_n3b1', \
            ]

# variables: tau1
features += ['FatJet_tau1', 'FatJet_DDX_tau1_flightDistance2dSig', 'FatJet_DDX_tau1_trackEtaRel_0', \
             'FatJet_DDX_tau1_trackEtaRel_1', 'FatJet_DDX_tau1_trackEtaRel_2', 'FatJet_DDX_tau1_trackSip3dSig_0', \
             'FatJet_DDX_tau1_trackSip3dSig_1', 'FatJet_DDX_tau1_vertexDeltaR', 'FatJet_DDX_tau1_vertexEnergyRatio', \
            ]

# variables: tau2
features += ['FatJet_tau2', 'FatJet_DDX_tau2_flightDistance2dSig', 'FatJet_DDX_tau2_trackEtaRel_0', \
             'FatJet_DDX_tau2_trackEtaRel_1', 'FatJet_DDX_tau2_trackEtaRel_3', 'FatJet_DDX_tau2_trackSip3dSig_0', \
             'FatJet_DDX_tau2_trackSip3dSig_1', 'FatJet_DDX_tau2_vertexEnergyRatio', \
            ]

# variables: tau3 and tau4
features += ['FatJet_tau3', 'FatJet_tau4',]

# variables: track
features += ['FatJet_DDX_trackSip2dSigAboveBottom_0', 'FatJet_DDX_trackSip2dSigAboveBottom_1', \
             'FatJet_DDX_trackSip2dSigAboveCharm', 'FatJet_DDX_trackSip3dSig_0', \
             'FatJet_DDX_trackSip3dSig_1', 'FatJet_DDX_trackSip3dSig_2', 'FatJet_DDX_trackSip3dSig_3', \
            ]

# variables: subjet 1
features += ['FatJet_subjet1_pt', 'FatJet_subjet1_eta', 'FatJet_subjet1_phi', \
             'FatJet_subjet1_Proba', 'FatJet_subjet1_tau1', 'FatJet_subjet1_tau2', \
             'FatJet_subjet1_tau3', 'FatJet_subjet1_tau4', 'FatJet_subjet1_n2b1', 'FatJet_subjet1_n3b1', \
            ]

# variables: subjet 2
features += ['FatJet_subjet2_pt', 'FatJet_subjet2_eta', 'FatJet_subjet2_phi', \
             'FatJet_subjet2_Proba', 'FatJet_subjet2_tau1', 'FatJet_subjet2_tau2', \
             'FatJet_subjet2_tau3', 'FatJet_subjet2_tau4', 'FatJet_subjet2_n2b1', 'FatJet_subjet2_n3b1', \
            ]

# variables: fatjet sv
features += ['FatJet_sv_costhetasvpv', 'FatJet_sv_d3dsig', 'FatJet_sv_deltaR', 'FatJet_sv_dxysig', \
             'FatJet_sv_enration', 'FatJet_sv_normchi2', 'FatJet_sv_ntracks', 'FatJet_sv_phirel', \
             'FatJet_sv_pt', 'FatJet_sv_ptrel', \
            ]

features = sorted(features)

# root_dir = "/eos/user/a/afriberg/datasets/QCD_samples/"

# dirs = os.listdir(root_dir)

### This gets QCD Data

In [4]:
# first_file = dirs.pop(0)
# while ".root" not in first_file:
#     first_file = dirs.pop(0)

# first_file = root_dir + first_file
# df = get_df(first_file, '*')
# # Select a particular type of particle 0 means QCD
# df.query("FatJet_gen_hadronFlavour == 0", inplace=True)
# df.dropna(inplace=True)
# df = df[features]
# # Prior to this, df is a pandas dataframe
# X = df.to_numpy().astype(np.float32)
# print(np.shape(X))


# for inputfile in dirs:
#     if ".root" not in inputfile:
#         continue
#     inputfile = root_dir + inputfile
#     df = get_df(inputfile, '*')
#     df.dropna(inplace=True)
#     df = df[features]
#     # Prior to this, df is a pandas dataframe
#     next_data = df.to_numpy().astype(np.float32)
#     print(f"next data has shape {np.shape(next_data)}")
#     # appending it to the whole thing
#     X = np.append(X, next_data, axis=0)

# print(f"X has shape {np.shape(X)}")

# Getting HToBB Data

In [5]:
new_path = "/eos/user/a/afriberg/datasets/has_B/ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8.root"
df = get_df(new_path, '*')
# hadron flavour of 4 means Charm, 5 means B quark. Check the paper David referenced for more information
# Getting bottom quark data
b_quarks = df.query("FatJet_gen_hadronFlavour == 5", inplace=False)
b_quarks = b_quarks.dropna(inplace=False)
b_quarks = b_quarks[features]

original_dim = len(b_quarks.dtypes)

b_quarks["is_B_quark"] = 1
# b_quarks.describe()

#### Doing the same for charm quark data

In [6]:
# Getting charm quark data
c_quarks = df.query("FatJet_gen_hadronFlavour == 4", inplace=False)
c_quarks = c_quarks.dropna(inplace=False)
c_quarks = c_quarks[features]

c_quarks["is_B_quark"] = 0

c_X = c_quarks.to_numpy()
c_X = c_X.astype("float32")

In [7]:
all_quarks = pd.concat([c_quarks, b_quarks])
# all_quarks.describe()

#### Turning it into a numpy array

In [8]:
X = all_quarks.to_numpy()
X = X.astype("float32")
print(np.shape(X))

(25353, 70)


In [9]:
b_quarks.describe()
# c_quarks.describe()

,FatJet_DDX_jetNSecondaryVertices,FatJet_DDX_jetNTracks,FatJet_DDX_tau1_flightDistance2dSig,FatJet_DDX_tau1_trackEtaRel_0,FatJet_DDX_tau1_trackEtaRel_1,FatJet_DDX_tau1_trackEtaRel_2,FatJet_DDX_tau1_trackSip3dSig_0,FatJet_DDX_tau1_trackSip3dSig_1,FatJet_DDX_tau1_vertexDeltaR,FatJet_DDX_tau1_vertexEnergyRatio,...,FatJet_sv_normchi2,FatJet_sv_ntracks,FatJet_sv_phirel,FatJet_sv_pt,FatJet_sv_ptrel,FatJet_tau1,FatJet_tau2,FatJet_tau3,FatJet_tau4,is_B_quark
count,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,...,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.000000,23350.0
mean,1.382441,10.302484,16.729216,2.335305,2.955847,2.219837,6.745109,-8.904252,-0.101879,1.826218,...,-5.020085,-2.985353,-6.318169,50.354596,-6.085274,0.194115,0.085310,0.063662,0.052793,1.0
std,1.038073,4.440896,26.264698,1.759065,2.139291,2.679875,33.228586,23.398584,0.429933,4.994916,...,24.612635,25.119014,24.215512,60.161516,24.276335,0.129628,0.047741,0.031345,0.026282,0.0
min,0.000000,0.000000,-190.750000,-1.000000,-1.000000,-1.000000,-50.000000,-50.000000,-1.000000,-1.000000,...,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,0.002764,0.000914,0.000000,0.000000,1.0
25%,1.000000,7.000000,2.529297,1.796875,2.214844,-1.000000,0.648438,-5.702148,0.005841,0.129639,...,0.281250,2.000000,-0.015068,19.812500,0.080261,0.083328,0.052406,0.041168,0.033691,1.0
50%,1.000000,10.000000,7.863281,2.728516,3.431641,3.123047,2.243164,0.528809,0.019791,0.625488,...,1.125000,3.000000,0.016129,47.968750,0.219604,0.155640,0.074097,0.058472,0.048615,1.0
75%,2.000000,13.000000,22.792969,3.562500,4.323242,4.371094,12.171875,1.973633,0.081482,1.752930,...,1.420898,4.000000,0.041382,89.125000,0.400635,0.290283,0.105469,0.080566,0.067749,1.0
max,8.000000,35.000000,525.000000,6.886719,9.484375,9.007812,1185.000000,288.500000,0.864258,50.000000,...,58.781250,7.000000,0.781738,241.875000,0.735840,0.640625,0.386475,0.272461,0.201416,1.0


This scales the HToBB data and the QCD data together, and then splits them in two parts afterwards

In [10]:
scaler = MinMaxScaler()
data = scaler.fit_transform(X)
print(f"data has shape {np.shape(data)}")
# Testing the normalized features
# test = pd.DataFrame(data, columns=features + ["is_B_quark"])
# test.describe()

data has shape (25353, 70)


## Run this regardless of presence or lack of HToBB data

In [11]:
training, testing = train_test_split(data, test_size=0.2)
x_train = training[:, :original_dim]
y_train = training[:, original_dim]

x_test = testing[:, :original_dim]
y_test = testing[:, original_dim]
# print(train_test_split(data, test_size=0.20)[:69])

In [12]:
batch_size = 32

def build_dset(df): 
    df = df.copy()
    dataset = tf.data.Dataset.from_tensor_slices((df, df))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset
    
x_train_dataset = build_dset(x_train)
x_test_dataset = build_dset(x_test)

### loss function

In [13]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### build model

In [14]:
def get_encoder(original_dim, latent_dim):
    # Encoder
    encoder_inputs = layers.Input(shape=(original_dim,))
    h = layers.Dense(32, activation='relu')(encoder_inputs)
    h = layers.Dense(16, activation='relu')(h)
    h = layers.Dense(8, activation='relu')(h)
    h = layers.Dense(latent_dim, activation='sigmoid')(h)
    z_mu = layers.Dense(latent_dim, name="z_mean")(h)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(h)
    z = Sampling()([z_mu, z_log_var])
    
    encoder = Model(encoder_inputs, [z_mu, z_log_var, z], name="encoder")
    return encoder
    
def get_decoder(original_dim, latent_dim):
    decoder_inputs = layers.Input(shape=(latent_dim,))
    d = layers.Dense(8, activation='relu')(decoder_inputs)
    d = layers.Dense(16, activation='relu')(d)
    d = layers.Dense(32, activation='relu')(d)
    d = layers.Dense(original_dim, activation='relu')(d)
    
    decoder = Model(decoder_inputs, d, name="decoder")
    return decoder

class vae(Model):
    def __init__(self, encoder, decoder, verbose=True, **kwargs):
        super(vae, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        if verbose:
            self.encoder.summary()
            self.decoder.summary()

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction), axis=-1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.encoder.trainable_weights + self.decoder.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.encoder.trainable_weights + self.decoder.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        y_pred = self.decoder(z)
        return y_pred 

### Building a classification model

In [15]:
class Binary(Model):
    # Takes in a PRE-TRAINED encoder. This is just a test, mostly
    def __init__(self, encoder, **kwargs):
        super(Binary, self).__init__(**kwargs)
        # Making a copy because we don't want to mess with the original
        self.encoder = tf.keras.models.clone_model(encoder)
        
        latent_length = self.encoder.layers[-1].output_shape[-1]
        for layer in self.encoder.layers:
            print(layer)
            layer.trainable = False
        
        # Some stuff that I honestly don't understand very well
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        
        # Creating a few classification layers
        class_inputs = layers.Input(shape=(latent_length,))
        h = layers.Dense(2, activation='relu')(class_inputs)
        h = layers.Dense(1, activation='sigmoid')(h)
        
        self.classifier = Model(class_inputs, h, name="classifier")
            
    def call(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        y_pred = self.classifier(z)

### JK we're making it a method cuz I don't know how classes work

In [16]:
def get_binary(latent_dim):
    input_layer = layers.Input(shape=(latent_dim,))
    h = layers.Dense(2, activation='relu')(input_layer)
    h = layers.Dense(1, activation='sigmoid')(h)
    binary =  Model(input_layer, h, name="classifier")
    return binary

In [17]:
classifier = get_binary(latent_dim)

# We have to get the output from the VAE to use in this model
encoder = model.encoder
z_mean, z_log_var, bin_x_train = encoder.predict(x_train)

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # parameters set to default values from Kingma and Ba paper.

print(np.shape(bin_x_train))
classifier.fit(bin_x_train, y_train, epochs=num_epochs)

NameError: name 'latent_dim' is not defined

# Train the VAE

In [ ]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.01
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 10
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

checkpoint_path = "checkpoints/roc_weights.{epoch:05d}.hdf5"
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor = 'val_loss',
                                                 save_weights_only=True,
                                                 save_best_only=False,
                                                 mode = 'min',
                                                 verbose=1)

num_epochs = 40

history = model.fit(
    x_train_dataset,
    shuffle=True,
    epochs=num_epochs,
    batch_size=batch_size,
    callbacks=[cp_callback]
    #callbacks=[cp_callback, get_lr_callback()]
)

# Loading a full VAE (or maybe AE) from a checkpoint

In [ ]:
latent_dim = 2
encoder = get_encoder(original_dim, latent_dim)
decoder = get_decoder(original_dim, latent_dim)
model = vae(encoder, decoder)
model.compile(optimizer=tf.keras.optimizers.Adam(1.e-3))
model.evaluate(x_test)
model.load_weights("working_QCD_checkpoints/tsg_vae weights.00020.hdf5")

# Plotting the test data errors

In [ ]:
predict = model.predict(x_test)
err = np.mean(np.abs(predict - x_test), axis=1)
print(np.shape(err))

bins = np.linspace(0, 0.4, 1000)
# plt.hist(err, density=True)
plt.hist(err, density=True, bins=bins)
plt.xlabel("Mean Absolute Error")
plt.ylabel("Number of events (density)")
# Getting the name of the file we ran on
plt.title("Aggregate QCD Data")
plt.show()


# Creating and training our binary classifier